# Machine Learning for Finance Freestyle

In this lab you'll be given the opportunity to apply everything you have learned to build a trading strategy for SP500 stocks. First, let's introduce the dataset you'll be using.

## The Data

Use BigQuery's magic function to pull data as follows:

    Dataset Name: ml4f
    Table Name: percent_change_sp500

The following query will pull 10 rows of data from the table:

In [ ]:
%%bigquery df
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
LIMIT
    10

In [ ]:
df.head()

As you can see, the table contains daily open and close data for SP500 stocks. The table also contains some features that have been generated for you using [navigation functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions) and [analytic functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts). Let's dig into the schema a bit more. 

In [10]:
%%bigquery 
SELECT
    * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
    `cloud-training-prod-bucket.ml4f`.INFORMATION_SCHEMA.COLUMNS
WHERE
    table_name = "percent_change_sp500"

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,cloud-training-prod-bucket,ml4f,percent_change_sp500,symbol,1,YES,STRING,NO,NO,NO,None
1,cloud-training-prod-bucket,ml4f,percent_change_sp500,Date,2,YES,DATE,NO,NO,NO,None
2,cloud-training-prod-bucket,ml4f,percent_change_sp500,Open,3,YES,FLOAT64,NO,NO,NO,None
3,cloud-training-prod-bucket,ml4f,percent_change_sp500,Close,4,YES,FLOAT64,NO,NO,NO,None
4,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomorrow_close,5,YES,FLOAT64,NO,NO,NO,None
5,cloud-training-prod-bucket,ml4f,percent_change_sp500,tomo_close_m_close,6,YES,FLOAT64,NO,NO,NO,None
6,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_5_days,7,YES,FLOAT64,NO,NO,NO,None
7,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_20_days,8,YES,FLOAT64,NO,NO,NO,None
8,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MIN_prior_260_days,9,YES,FLOAT64,NO,NO,NO,None
9,cloud-training-prod-bucket,ml4f,percent_change_sp500,close_MAX_prior_5_days,10,YES,FLOAT64,NO,NO,NO,None


Most of the features, like `open` and `close` are pretty straightforward. The features generated using analytic functions, such as `close_MIN_prior_5_days` are best described using an example. Let's take the 6 most recent rows of data for IBM and reproduce the `close_MIN_prior_5_days` column. 

In [ ]:
%%bigquery
SELECT 
    *
FROM
    `cloud-training-prod-bucket.ml4f.percent_change_sp500`
WHERE
    symbol = 'IBM'
ORDER BY 
    Date DESC
LIMIT 6

For `Date = 2013-02-01` how did we arrive at `close_MIN_prior_5_days = 0.989716`? The minimum close over the past five days was `203.07`. This is normalized by the current day's close of `205.18` to get `close_MIN_prior_5_days = 203.07 / 205.18 = 0.989716`. The other features utilizing analytic functions were generated in a similar way. Here are explanations for some of the other features:

* __scaled_change__: `tomo_close_m_close / close`
* __s_p_scaled_change__: This value is calculated the same way as `scaled_change` but for the S&P 500 index. 
* __normalized_change__: `scaled_change - s_p_scaled_change` The normalization using the S&P index fund helps ensure that the future price of a stock is not due to larger market effects. Normalization helps us isolate the factors contributing to the performance of a stock_market.
* __direction__: This is the target variable we're trying to predict. The logic for this variable is as follows: 

    ```sql
    CASE 
        WHEN normalized_change < -0.01 THEN 'DOWN'
        WHEN normalized_change > 0.01 THEN 'UP'
        ELSE 'STAY'
    END AS direction
    ```

## Create classification model for `direction`

In this example, your job is to create a classification model to predict the `direction` of each stock. Be creative! You can do this in any number of ways. For example, you can use BigQuery, Scikit-Learn, or AutoML. Feel free to add additional features, or use time series models.   



### Establish a Simple Benchmark

One way to assess the performance of a model is to compare it to a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicting the majority class. Across the entire dataset, the majority class is 'STAY'. Using the following query we can see how this naive strategy would perform.

In [9]:
%%bigquery
WITH subset as (
    SELECT 
        Direction
    FROM
        `cloud-training-prod-bucket.ml4f.percent_change_sp500`
    WHERE
        tomorrow_close IS NOT NULL
)
SELECT 
    Direction,
    100.0 * COUNT(*) / (SELECT COUNT(*) FROM subset) as percentage
FROM
    subset
GROUP BY
    Direction

,Direction,percentage
0,STAY,53.766049
1,UP,23.240681
2,DOWN,22.993271


So, the naive strategy of just guessing the majority class would have accuracy of around 54% across the entire dataset. See if you can improve on this. 

### Train Your Own Model

In [16]:
# TODO: Write code to build a model to predict Direction